# Starter code for submitting jobs with AWS SageMaker




Import the necessary packages

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role
import boto3
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name
print(role, region)

arn:aws:iam::986428434207:role/analysis-1594681887916-sagemaker-notebook-role ap-southeast-2


In [ ]:
# train_data_bucket_name = 's3://sagemaker-test-ml-models/mnist/'

# !aws s3 cp 'train_data.npy' 's3://sagemaker-test-ml-models/mnist/train_data.npy' 
# !aws s3 cp 'train_labels.npy' 's3://sagemaker-test-ml-models/mnist/train_labels.npy' 
# !aws s3 cp 'eval_data.npy' 's3://sagemaker-test-ml-models/mnist/eval_data.npy' 
# !aws s3 cp 'eval_labels.npy' 's3://sagemaker-test-ml-models/mnist/eval_labels.npy' 

# !aws s3 cp 's3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_data.npy' './Data/train_data.npy'
# !aws s3 cp 's3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_labels.npy' './Data/train_labels.npy'
# !aws s3 cp 's3://sagemaker-sample-data-us-east-1/tensorflow/mnist/eval_data.npy' './Data/eval_data.npy'
# !aws s3 cp 's3://sagemaker-sample-data-us-east-1/tensorflow/mnist/eval_labels.npy' './Data/eval_labels.npy'

Copy the training files to s3 bucket. Should use the student number as the prefix

In [5]:
bucket = 'sagemaker-ap-southeast-2-986428434207'
prefix = 's9999999'     #THIS SHOULD BE YOUR STUDENT NUMBER

def write_to_s3(file, bucket, prefix):
    return boto3.Session(region_name=region).resource('s3').Bucket(bucket).Object(prefix).upload_file(file)

write_to_s3('../Data/train_data.npy', bucket, f'{prefix}/mnist_data/train_data.npy')
write_to_s3('../Data/train_labels.npy', bucket, f'{prefix}/mnist_data/train_labels.npy')
write_to_s3('../Data/eval_data.npy', bucket, f'{prefix}/mnist_data/eval_data.npy')
write_to_s3('../Data/eval_labels.npy', bucket, f'{prefix}/mnist_data/eval_labels.npy')

training_input_path = os.path.join("s3://", bucket, prefix, "mnist_data")
print(training_input_path)

s3://sagemaker-ap-southeast-2-986428434207/s9999999/mnist_data


Create the model deployment instance

In [6]:
mnist_estimator2 = TensorFlow(entry_point='mnist_2.py',
                             role=role,
                             train_instance_count=2,
                             train_instance_type='ml.p2.xlarge',
                             framework_version='2.1.0',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}},
                             output_path='s3://sagemaker-ap-southeast-2-986428434207',
                             base_job_name='s9999999')

Parameter distribution will be renamed to {'parameter_server': {'enabled': True}} in SageMaker Python SDK v2.


Run the training job

In [7]:
mnist_estimator2.fit(training_input_path)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-07-14 00:40:29 Starting - Starting the training job...
2020-07-14 00:40:31 Starting - Launching requested ML instances......
2020-07-14 00:41:37 Starting - Preparing the instances for training......
2020-07-14 00:42:54 Downloading - Downloading input data...
2020-07-14 00:43:09 Training - Downloading the training image........2020-07-14 00:44:38,780 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-07-14 00:44:39,014 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2020-07-14 00:44:39,014 sagemaker_tensorflow_container.training INFO     Launching parameter server process
2020-07-14 00:44:39,014 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2020-07-14 00:44:39,617 sagemaker_tensorflow_container.training INFO     Launching worker process
2020-07-14 00:44:39,937 sagemaker-containers INFO     Invoking user script

Training Env:


In [11]:
output_bucket_path = 's3://sagemaker-ap-southeast-2-986428434207/s9999999-2020-07-14-00-40-29-447/output/'
!aws s3 ls $output_bucket_path

2020-07-14 00:46:09   16271192 model.tar.gz


In [12]:
output_file_path = output_bucket_path + 'model.tar.gz'
!aws s3 cp $output_file_path '../Outputs/model.tar.gz'

download: s3://sagemaker-ap-southeast-2-986428434207/s9999999-2020-07-14-00-40-29-447/output/model.tar.gz to ../Outputs/model.tar.gz


In [15]:
!tar -zxvf ../Outputs/model.tar.gz -C ../Outputs/

my_model.h5
my_model.h5


In [1]:
from mnist_2 import _load_testing_data

x_test, y_test = _load_testing_data('../Data/')

In [2]:
from mnist_2 import evalaute_model

evalaute_model(x_test, y_test, '../Outputs/my_model.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  803840    
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  10250     
Total params: 814,090
Trainable params: 814,090
Non-trainable params: 0
_________________________________________________________________
10000/10000 [==============================] - 1s 105us/sample - loss: 0.1171 - accuracy: 0.9623
